In [36]:
# read the data
#imports"
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "database.csv"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

Current Directory: C:\Users\33783\Documents\EPFL\Etudes\BA4\Amazing-Project
AAM,ea



In [53]:
# function to convert name 
!pip install pubchempy

import pubchempy as pcp

def name_to_smiles(molecule_name):
    """
    Convert a molecule name to a SMILES notation using PubChemPy's PubChem database.
    Args:
    - molecule_name (str): The name of the molecule.
    Returns:
    - smiles (str): The SMILES notation of the molecule, or None if retrieval fails.
    """
    try:
        compound = pcp.get_compounds(molecule_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            print("Error: Unable to retrieve molecule information.")
            return None
    except:
        print("Error: Unable to retrieve molecule information.")
        return None


In [54]:
from rdkit import Chem

def is_smiles(smiles):
    """
    Check if a string represents a valid SMILES notation.
    Args:
    - smiles (str): The string to check.
    Returns:
    - is_valid (bool): True if the string is a valid SMILES notation, False otherwise.
    """
    try:
        Chem.MolFromSmiles(smiles)
        return True
    except:
        return False

# Loop to continuously prompt the user for input
while True:
    # Prompt the user to enter a string
    string_input = input("Enter a string (or 'exit' to quit): ")

    # Check if the user wants to exit
    if string_input.lower() == 'exit':
        print("Exiting the program.")
        break

    # Check if the entered string is a valid SMILES notation
    if is_smiles(string_input):
        print(f"'{string_input}' is a valid SMILES notation.")
        break
    else:
        print(f"'{string_input}' is not a valid SMILES notation. Please try again.")

Enter a string (or 'exit' to quit):  exit


Exiting the program.


In [55]:
# Prompt the user to enter a string
string_input = input("Enter a string: ")

Enter a string:  aspirin


In [59]:
# Call the function to convert the molecule name to SMILES notation

if is_smiles(string_input)==0:
    smiles = name_to_smiles(string_input)
    if smiles:
        print(f"SMILES notation for {string_input}: {smiles}")
else:
    print (string_input)

aspirin


[17:01:24] SMILES Parse Error: syntax error while parsing: aspirin
[17:01:24] SMILES Parse Error: Failed parsing SMILES 'aspirin' for input: 'aspirin'


In [ ]:
# print the yield

In [41]:
# print the molecule
original_smiles="[Br-:1].[Br:2][C:5]([C:4]([H:3])([C:31]#[N:32])[C:41]#[N:42])([H:11])[H:21]"
# Parse the original SMILES notation
mol = Chem.MolFromSmiles(original_smiles)

# Convert the parsed molecule back to SMILES without atom mapping numbers
smiles_without_numbers = Chem.MolToSmiles(mol, allBondsExplicit=True, isomericSmiles=True)


# Create a molecule object from the SMILES notation
mol = Chem.MolFromSmiles(smiles_without_numbers)

# Draw and display the molecule
img = Draw.MolToImage(mol)

# Display the image
img.show()